# T-LSTM Hyperparameter Search Notebook
This notebook implements a Time-Aware LSTM for predicting the next tab event, with a grid search over hyperparameters.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# ——— Data Loading & Filtering from your snippet ———
df = pd.read_csv(
    "./user_data_qasim_1.csv",
    header=None,
    on_bad_lines='skip',
    encoding="cp1252"
)
df.columns = ["timestamp", "type", "data"]
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df = df.dropna(subset=['timestamp'])
df = df[
    (df['type'] != 'memoryUsage') & 
    (df['type'] != 'tabDuration') &
    (df['type'] != 'resourceUsage') &
    (df['type'] != 'periodicBrowserStats')
]
df = df.dropna()
df.sort_values("timestamp", inplace=True)

# ——— Compute inter-event deltas ———
df['delta'] = df['timestamp'].diff().dt.total_seconds().fillna(0)

# ——— Encode and scale ———
le = LabelEncoder()
df['event_id'] = le.fit_transform(df['type'])
scaler = MinMaxScaler()
df['delta_scaled'] = scaler.fit_transform(df[['delta']])

# ——— Build sequences for T-LSTM ———
LOOKBACK = 15
events = df['event_id'].values
deltas = df['delta_scaled'].values

X_events, X_deltas, y = [], [], []
for i in range(len(events) - LOOKBACK):
    X_events.append(events[i:i+LOOKBACK])
    X_deltas.append(deltas[i:i+LOOKBACK].reshape(-1, 1))
    y.append(events[i+LOOKBACK])
X_events = np.array(X_events)
X_deltas = np.array(X_deltas)
y = np.array(y)

num_classes = len(le.classes_)

df.head()  # Display the first few rows of the DataFrame

,timestamp,type,data,delta,event_id,delta_scaled
16,2025-03-10 23:22:26,tabSwitched,"{'type': 'tabSwitched', 'fromTab': None, 'toTa...",0.0,5,0.000000
17,2025-03-10 23:22:26,tabHighlighted,"{'type': 'tabHighlighted', 'windowId': 8379253...",0.0,3,0.000000
19,2025-03-10 23:22:27,tabSwitched,"{'type': 'tabSwitched', 'fromTab': 837925578, ...",1.0,5,0.000054
20,2025-03-10 23:22:27,tabHighlighted,"{'type': 'tabHighlighted', 'windowId': 8379253...",0.0,3,0.000000
28,2025-03-10 23:22:57,tabCreated,"{'type': 'tabCreated', 'tabId': 837925613, 'ur...",30.0,1,0.001622


In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate
from tensorflow.keras.optimizers import Adam

def build_tlstm_model(seq_len, vocab_size, embed_dim, lstm_units, learning_rate):
    # Inputs
    events_in = Input(shape=(seq_len,), name='events')
    deltas_in = Input(shape=(seq_len, 1), name='deltas')
    # Embedding and concat
    x = Embedding(vocab_size, embed_dim, name='embed')(events_in)
    x = Concatenate(name='concat')([x, deltas_in])
    # LSTM
    x = LSTM(lstm_units, name='lstm')(x)
    out = Dense(vocab_size, activation='softmax', name='output')(x)
    model = Model([events_in, deltas_in], out)
    model.compile(
        optimizer=Adam(learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

/Users/muhammadqasimatiqullah/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
import tensorflow.keras.backend as K

# Define your grid
param_grid = {
    'embed_dim':    [8, 16],
    'lstm_units':   [16, 32],
    'learning_rate':[1e-3, 5e-4],
    'batch_size':   [32, 64],
    'epochs':       [10, 20]
}

best_loss = np.inf
best_params = None

for embed_dim in param_grid['embed_dim']:
    for lstm_units in param_grid['lstm_units']:
        for lr in param_grid['learning_rate']:
            for batch_size in param_grid['batch_size']:
                for epochs in param_grid['epochs']:
                    K.clear_session()
                    model = build_tlstm_model(
                        LOOKBACK, num_classes,
                        embed_dim, lstm_units, lr
                    )
                    history = model.fit(
                        [X_events, X_deltas], y,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_split=0.2,
                        verbose=0
                    )
                    val_loss = history.history['val_loss'][-1]
                    print(f"embed={embed_dim}, units={lstm_units}, "
                          f"lr={lr}, bs={batch_size}, ep={epochs} "
                          f"→ val_loss={val_loss:.4f}")
                    if val_loss < best_loss:
                        best_loss = val_loss
                        best_params = (embed_dim, lstm_units, lr, batch_size, epochs)

print("\nBest configuration:", best_params, "with val_loss=", best_loss)

embed=8, units=16, lr=0.001, bs=32, ep=10 → val_loss=1.1168
embed=8, units=16, lr=0.001, bs=32, ep=20 → val_loss=1.0691
embed=8, units=16, lr=0.001, bs=64, ep=10 → val_loss=1.2021
embed=8, units=16, lr=0.001, bs=64, ep=20 → val_loss=1.0816
embed=8, units=16, lr=0.0005, bs=32, ep=10 → val_loss=1.2374
embed=8, units=16, lr=0.0005, bs=32, ep=20 → val_loss=1.1495
embed=8, units=16, lr=0.0005, bs=64, ep=10 → val_loss=1.2874
embed=8, units=16, lr=0.0005, bs=64, ep=20 → val_loss=1.1556
embed=8, units=32, lr=0.001, bs=32, ep=10 → val_loss=1.0848
embed=8, units=32, lr=0.001, bs=32, ep=20 → val_loss=1.0420
embed=8, units=32, lr=0.001, bs=64, ep=10 → val_loss=1.1263
embed=8, units=32, lr=0.001, bs=64, ep=20 → val_loss=1.0647
embed=8, units=32, lr=0.0005, bs=32, ep=10 → val_loss=1.2061
embed=8, units=32, lr=0.0005, bs=32, ep=20 → val_loss=1.0820
embed=8, units=32, lr=0.0005, bs=64, ep=10 → val_loss=1.2840
embed=8, units=32, lr=0.0005, bs=64, ep=20 → val_loss=1.1213
embed=16, units=16, lr=0.001, bs

**Instructions**: Save this notebook as `tlstm_hyperparam_notebook.ipynb` in the same directory as your data and Jupyter file. Then run it to perform hyperparameter search for the Time-Aware LSTM.

In [4]:
import pickle
import numpy as np
import tensorflow.keras.backend as K

# 1) Unpack your best hyperparameters
embed_dim, lstm_units, lr, batch_size, epochs = best_params

# 2) (Re)build and retrain on the full dataset
K.clear_session()
best_model = build_tlstm_model(
    seq_len=LOOKBACK,
    vocab_size=num_classes,
    embed_dim=embed_dim,
    lstm_units=lstm_units,
    learning_rate=lr
)
best_model.fit(
    [X_events, X_deltas],
    y,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1
)

print(" ======================================= ")

#3) Run with Validation Level
K.clear_session()
best_model = build_tlstm_model(
    seq_len=LOOKBACK,
    vocab_size=num_classes,
    embed_dim=embed_dim,
    lstm_units=lstm_units,
    learning_rate=lr
)
history = best_model.fit(
    [X_events, X_deltas],
    y,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,   # reserve 20% of data for validation
    verbose=1
)


best_model.save('tlstm_full.h5')

# 3) Bundle config, weights, and your LabelEncoder classes
model_bundle = {
    'config':  best_model.get_config(),
    'weights': best_model.get_weights(),
    'classes': le.classes_.tolist()
}

# # 4) Write out the pickle
# with open('tlstm_best_model.pkl', 'wb') as f:
#     pickle.dump(model_bundle, f)

# print("Saved Time-Aware LSTM model to tlstm_best_model.pkl")

Epoch 1/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3537 - loss: 2.0557
Epoch 2/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4947 - loss: 1.3729
Epoch 3/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5958 - loss: 1.1906
Epoch 4/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6182 - loss: 1.1288
Epoch 5/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6223 - loss: 1.1082
Epoch 6/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6343 - loss: 1.0618
Epoch 7/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6444 - loss: 1.0526
Epoch 8/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6486 - loss: 1.0292
Epoch 9/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6509 - loss: 1.0383
Epoch 10/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6646 - loss: 0.9982
Epoch 11/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6548 - loss: 1.0179
Epoch 12/20
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

In [5]:
# pick the first example from your data
idx = 0

# raw inputs
sample_events = X_events[idx]         # shape (LOOKBACK,)
sample_deltas = X_deltas[idx].flatten()  # shape (LOOKBACK,)

# decode to human‐readable event names
event_names = le.inverse_transform(sample_events)

print("▶ Past events sequence:")
for i, (e, dt) in enumerate(zip(event_names, sample_deltas)):
    print(f"  t-{LOOKBACK-i}: event='{e}', Δt={dt:.3f}")

▶ Past events sequence:
  t-15: event='tabSwitched', Δt=0.000
  t-14: event='tabHighlighted', Δt=0.000
  t-13: event='tabSwitched', Δt=0.000
  t-12: event='tabHighlighted', Δt=0.000
  t-11: event='tabCreated', Δt=0.002
  t-10: event='tabSwitched', Δt=0.000
  t-9: event='windowFocused', Δt=0.000
  t-8: event='tabHighlighted', Δt=0.000
  t-7: event='tabUpdated', Δt=0.000
  t-6: event='tabTitleChanged', Δt=0.000
  t-5: event='tabSwitched', Δt=0.000
  t-4: event='tabHighlighted', Δt=0.000
  t-3: event='tabRemoved', Δt=0.001
  t-2: event='tabHighlighted', Δt=0.000
  t-1: event='tabHighlighted', Δt=0.000


In [6]:
# get T-LSTM prediction distribution
probs_t = best_model.predict(
    [sample_events.reshape(1,-1), sample_deltas.reshape(1,LOOKBACK,1)]
)[0]

# top-3 events
top3_idx = probs_t.argsort()[-3:][::-1]
top3 = list(zip(
    le.inverse_transform(top3_idx),
    probs_t[top3_idx]
))

print("▶ T-LSTM top-3 predictions:")
for event, p in top3:
    print(f"  {event:<20}  p={p:.3f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
▶ T-LSTM top-3 predictions:
  tabSwitched           p=0.827
  tabUpdated            p=0.053
  windowFocused         p=0.039
